In [18]:
import transformers
import pandas as pd
import torch
from huggingface_hub import notebook_login
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
import os

In [12]:
list_sq = ["AI will be led by companies like Tesla.",
            "AI could be a risk for many workers.",
            "I am sure that social networks like facebook should use AI to counteract hatespeech comments.",
            "In the future, cars will be powered by alternative energy sources such as solar and wind power.",
            "In the future, there will be driverless cars.",
            "In the future, flying cars will be a common form of transport.",
            "In the future, hoverboards will be a common form of transport.",
            "In the future, the use of public transport will increase as it becomes more efficient and affordable.",
            "In the future, the number of people working from home will increase as technology advances.",
            "In the future, there will be more jobs in the service sector as automation increases.",
            "In the future, there will be more jobs in the green economy as we move away from fossil fuels.",
            "In the future, artificial intelligence will lead to the creation of new jobs in fields such as healthcare and finance.",
            "In the future, social media will be used more for business networking and less for personal use.",
            "In the future, there will be more regulation of social media to protect people's privacy.",
            "In the future, virtual reality will be used more for social interaction and entertainment.",
            "In the future, augmented reality will be used"
            ]

## Topics

In [6]:
#topic classifier
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")
#topic classifier - labeling
candidate_labels = ['Labour market', 'transport', 'social media']
cl = classifier(list_sq, candidate_labels, multi_class=False)

The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.


In [7]:
for i in range(len(cl)):
    for k in cl[i]:
        print(cl[i][k])

AI will be led by companies like Tesla.
['Labour market', 'transport', 'social media']
[0.5151784420013428, 0.3396190404891968, 0.14520250260829926]
AI could be a risk for many workers.
['Labour market', 'social media', 'transport']
[0.7350534200668335, 0.14033888280391693, 0.12460768222808838]
I am sure that social networks like facebook should use AI to counteract hatespeech comments.
['social media', 'transport', 'Labour market']
[0.9912010431289673, 0.005565676372498274, 0.0032333184499293566]
In the future, cars will be powered by alternative energy sources such as solar and wind power.
['transport', 'Labour market', 'social media']
[0.9687971472740173, 0.02224564738571644, 0.008957191370427608]
In the future, there will be driverless cars.
['transport', 'Labour market', 'social media']
[0.9697248339653015, 0.016401909291744232, 0.013873287476599216]
In the future, flying cars will be a common form of transport.
['transport', 'Labour market', 'social media']
[0.980880081653595, 0.

In [8]:
sequences_list = []
topics_list = []
scores_topics_list = []
for i in range(len(cl)):
    sequences_list.append(cl[i]['sequence'])
    topics_list.append(cl[i]['labels'][0])
    scores_topics_list.append(cl[i]['scores'][0])
#print(sequences_list)
#print(topics_list)
#print(scores_topics_list)
df = pd.DataFrame(data={'sequence':sequences_list, 'labels':topics_list, 'scores':scores_topics_list})

In [9]:
df

,sequence,labels,scores
0,AI will be led by companies like Tesla.,Labour market,0.515178
1,AI could be a risk for many workers.,Labour market,0.735053
2,I am sure that social networks like facebook s...,social media,0.991201
3,"In the future, cars will be powered by alterna...",transport,0.968797
4,"In the future, there will be driverless cars.",transport,0.969725
5,"In the future, flying cars will be a common fo...",transport,0.980880
6,"In the future, hoverboards will be a common fo...",transport,0.979451
7,"In the future, the use of public transport wil...",transport,0.985049
8,"In the future, the number of people working fr...",Labour market,0.584065
9,"In the future, there will be more jobs in the ...",Labour market,0.821424
